In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
import time

# Set up the WebDriver (choose your preferred browser)
service = Service(executable_path="chromedriver.exe")
options = webdriver.ChromeOptions()



# Add other desired options here

driver = webdriver.Chrome(options=options)
driver.get('https://www.bls.gov/ooh/home.htm')
driver.maximize_window()
time.sleep(5)



In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import pandas as pd
import time

# Set up the WebDriver (choose your preferred browser)
service = Service(executable_path="chromedriver.exe")
options = webdriver.ChromeOptions()
# Add other desired options here

# Start the WebDriver
driver = webdriver.Chrome(service=service, options=options)

# Open the webpage
driver.get('https://www.bls.gov/ooh/home.htm')

# Wait for some time to ensure the page is fully loaded
time.sleep(5)

# Get the page source
page_source = driver.page_source

# Parse the page source using BeautifulSoup
soup = BeautifulSoup(page_source, 'html.parser')

# Find all <a> tags with href containing '/ooh/'
links = soup.find_all('a', href=lambda href: href and '/ooh/' in href)

# Extract the href and text of each link
link_data = []
for link in links:
    link_data.append({'href': link['href'], 'text': link.text.strip()})

# Convert the list of dictionaries to a pandas DataFrame
df = pd.DataFrame(link_data)

# Close the WebDriver
driver.quit()

# Display the DataFrame
print(df)



In [ ]:
# Save the DataFrame to a CSV file
df.to_csv('links.csv', index=False)

In [37]:
import pandas as pd
pd_links = pd.read_csv('links.csv')
pd_links = pd_links[~pd_links['text'].str.contains('view', na=False)]
# Drop rows with index from 0 to 9
pd_links.drop(index=pd_links.index[:9], inplace=True)

# Drop rows with index from 700 to 750
pd_links.drop(index=pd_links.index[358:], inplace=True)
# Add a new column named 'type' and set initial value as 'occupation'
pd_links['type'] = 'occupation'

# Mark the first 23 rows as 'group' in the 'type' column
pd_links.loc[:33, 'type'] = 'group'
pd_links['grp'] = pd_links['href'].str.split('/').str[2]

vl = pd_links.iloc[:25, [1, 3]].copy()

pd_links = pd_links.merge(vl, on='grp', how='left')

pd_links.rename(columns={'text_y': 'group'}, inplace=True)
pd_links.rename(columns={'text_x': 'Occupation'}, inplace=True)